In [ ]:
import pandas as pd

store_df=pd.read_pickle("../data/hyundai_store.pkl")

In [5]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   agencyCode       719 non-null    object 
 1   agencyName       719 non-null    object 
 2   agencyTypeCode   719 non-null    object 
 3   agencyAddress    719 non-null    object 
 4   agencyTel        719 non-null    object 
 5   lattitude        719 non-null    object 
 6   longitude        719 non-null    object 
 7   distance         719 non-null    object 
 8   displayCarCount  719 non-null    int64  
 9   carMasterCount   719 non-null    int64  
 10  distance_km      719 non-null    float64
 11  lat              719 non-null    float64
 12  lon              719 non-null    float64
 13  is_showroom      719 non-null    bool   
 14  agencyType       719 non-null    object 
 15  sido_code        719 non-null    object 
 16  gugun_code       719 non-null    object 
dtypes: bool(1), floa

In [6]:
store_df.head()

,agencyCode,agencyName,agencyTypeCode,agencyAddress,agencyTel,lattitude,longitude,distance,displayCarCount,carMasterCount,distance_km,lat,lon,is_showroom,agencyType,sido_code,gugun_code
0,DB39004,도곡중앙,2,서울특별시 강남구 언주로 122 (도곡동) (06295),02-3463-5200,37.487082477134,127.051017587398,2.6,0,18,2.6,37.487082,127.051018,False,대리점,01,0101
1,B39,강남대로,1,서울특별시 강남구 강남대로 292 현대자동차(주) 강남대로지점 전시장 (06258),02-3461-1990,37.4893966742,127.032205066703,2.9,8,36,2.9,37.489397,127.032205,True,지점/전시장,01,0101
2,DB44007,언주,2,서울특별시 강남구 도곡로 251 1층 전시장 2층 사무실 (06230),02-6203-8200,37.494250167536,127.045747575968,3.3,0,15,3.3,37.494250,127.045748,False,대리점,01,0101
3,DB44006,신대치,2,서울특별시 강남구 삼성로 324 (대치동) (06191),02-556-8777,37.500837553551,127.060137997529,4.3,0,18,4.3,37.500838,127.060138,False,대리점,01,0101
4,X83,대치,1,"서울특별시 강남구 영동대로 302 (구상빌딩1,2층) (06177)",02-564-0365,37.501514402321,127.067769302362,4.6,3,24,4.6,37.501514,127.067769,True,지점/전시장,01,0101


### 지도 시각화 진행

In [17]:
import pandas as pd
import plotly.graph_objects as go
import json

hovertemplate = """
<div style="font-size:14px; line-height:1.6;">
  <b style="font-size:16px;">%{text}</b><br>
  <span style="color:#888;">%{customdata[0]}</span>
  <hr style="margin:6px 0;">
  🚗 <b>유형</b> : %{customdata[1]}<br>
  📍 <b>거리</b> : %{customdata[2]:.1f} km<br>
  🏷 <b>전시장 차량</b> : %{customdata[3]}대<br>
  📦 <b>보유 차량</b> : %{customdata[4]}대<br>
  <br>
  🏠 <b>주소</b><br>
  <span style="font-size:13px;">%{customdata[5]}</span><br>
  ☎ <b>전화</b> : %{customdata[6]}
</div>
<extra></extra>
"""



dff = store_df.copy()
dff["lat"] = pd.to_numeric(dff["lat"], errors="coerce")
dff["lon"] = pd.to_numeric(dff["lon"], errors="coerce")
dff = dff.dropna(subset=["lat", "lon"])

raw_json_str = """{"data":[
{"code":"01","codeName":"서울특별시"},
{"code":"06","codeName":"인천광역시"},
{"code":"07","codeName":"경기도"},
{"code":"02","codeName":"강원특별자치도"},
{"code":"05","codeName":"충청남도"},
{"code":"03","codeName":"대전광역시"},
{"code":"04","codeName":"충청북도"},
{"code":"15","codeName":"대구광역시"},
{"code":"16","codeName":"경상북도"},
{"code":"11","codeName":"부산광역시"},
{"code":"13","codeName":"경상남도"},
{"code":"12","codeName":"울산광역시"},
{"code":"18","codeName":"전북특별자치도"},
{"code":"10","codeName":"전라남도"},
{"code":"08","codeName":"광주광역시"},
{"code":"14","codeName":"제주특별자치도"}
], "rspStatus":{"rspCode":"0000","rspMessage":"성공","uri":""}}"""

obj = json.loads(raw_json_str)
sido_map = {x["code"]: x["codeName"] for x in obj["data"]}
dff["sido_name"] = dff["sido_code"].map(sido_map).fillna(dff["sido_code"])


# 색 지정
type_color = {
    "지점/전시장": "#e63946",
    "대리점": "#457b9d",
}

def add_traces(fig, data, name_prefix=""):
    """data 안에서 agencyType별로 trace 2개 추가"""
    for t, color in type_color.items():
        sub = data[data["agencyType"] == t]
        fig.add_trace(
           go.Scattermapbox(
                lat=sub["lat"],
                lon=sub["lon"],
                mode="markers",
                marker=dict(size=9, color=color),
                text=sub["agencyName"],
                customdata=sub[
                    [
                        "agencyName",
                        "agencyType",
                        "distance_km",
                        "displayCarCount",
                        "carMasterCount",
                        "agencyAddress",
                        "agencyTel",
                    ]
                ].values,
                hovertemplate=hovertemplate,
                name=f"{name_prefix}{t}",
            )
        )

fig = go.Figure()

# ✅ 1) 전체(유형별 2개 trace)
add_traces(fig, dff, name_prefix="전체-")

# ✅ 2) 시도별(각 시도마다 유형별 2개 trace) 만들고 처음엔 숨김
# sido_name 없으면 sido_code로
sido_col = "sido_name" if "sido_name" in dff.columns else "sido_code"
sido_list = sorted(dff[sido_col].unique())

start_idx = len(fig.data)  # 전체 trace 2개 다음부터
for s in sido_list:
    sub_s = dff[dff[sido_col] == s]
    # 이 시도 trace들은 처음엔 숨김
    before = len(fig.data)
    add_traces(fig, sub_s, name_prefix=f"{s}-")
    for k in range(before, len(fig.data)):
        fig.data[k].visible = False

# dropdown 버튼 만들기
buttons = []

# 전체 버튼: 처음 2개만 True
vis_all = [False] * len(fig.data)
vis_all[0] = True
vis_all[1] = True
buttons.append(dict(label="전체", method="update", args=[{"visible": vis_all}]))

# 시도 버튼: 해당 시도에 해당하는 2개 trace만 True
# 시도마다 trace 2개씩 추가됨
for i, s in enumerate(sido_list):
    vis = [False] * len(fig.data)
    # 전체 2개는 끄고
    # 시도 i의 시작 인덱스 = start_idx + i*2
    base = start_idx + i * 2
    vis[base] = True
    vis[base + 1] = True
    buttons.append(dict(label=str(s), method="update", args=[{"visible": vis}]))

fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        zoom=9,
        center=dict(lat=dff["lat"].mean(), lon=dff["lon"].mean()),
    ),
    width=1600,
    height=550,
    margin=dict(l=10, r=10, t=60, b=10),
    title="현대차 대리점/전시장 지도",
    updatemenus=[dict(
        type="dropdown",
        x=0.01, y=0.99,
        xanchor="left", yanchor="top",
        buttons=buttons
    )],
    hoverlabel=dict(
    bgcolor="white",
    font_size=14,
    font_family="Pretendard, Apple SD Gothic Neo, Malgun Gothic",
    bordercolor="#ddd",
    align="left",
    ),   

    legend=dict(orientation="h", y=1.02, x=1, xanchor="right", yanchor="bottom"),
)

fig.show(config={"scrollZoom": True})


C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\1154956110.py:64: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\1154956110.py:64: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\1154956110.py:64: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\1154956110.py:64: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\1154956110.py:64: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https:/

In [18]:
import pandas as pd
import plotly.graph_objects as go

# =========================
# 1) 설정값 / 매핑
# =========================
SIDO_MAP = {
    "01": "서울특별시",
    "06": "인천광역시",
    "07": "경기도",
    "02": "강원특별자치도",
    "05": "충청남도",
    "03": "대전광역시",
    "04": "충청북도",
    "15": "대구광역시",
    "16": "경상북도",
    "11": "부산광역시",
    "13": "경상남도",
    "12": "울산광역시",
    "18": "전북특별자치도",
    "10": "전라남도",
    "08": "광주광역시",
    "14": "제주특별자치도",
}

TYPE_COLOR = {
    "지점/전시장": "#e63946",
    "대리점": "#457b9d",
}

# ✅ HTML 태그 최소화 (div/span/hr 제거)
HOVER_TEMPLATE = (
    "<b>%{text}</b><br>"
    "유형: %{customdata[0]}<br>"
    "거리: %{customdata[1]:.1f} km<br>"
    "전시장 차량: %{customdata[2]}대<br>"
    "보유 차량: %{customdata[3]}대<br>"
    "<br>"
    "주소: %{customdata[4]}<br>"
    "전화: %{customdata[5]}"
    "<extra></extra>"
)

# =========================
# 2) 데이터 전처리
# =========================
dff = store_df.copy()

dff["lat"] = pd.to_numeric(dff["lat"], errors="coerce")
dff["lon"] = pd.to_numeric(dff["lon"], errors="coerce")
dff["distance_km"] = pd.to_numeric(dff["distance_km"], errors="coerce")

dff = dff.dropna(subset=["lat", "lon"])
dff["sido_name"] = dff["sido_code"].map(SIDO_MAP).fillna(dff["sido_code"])

# 시도 컬럼(있으면 이름, 없으면 코드)
SIDO_COL = "sido_name" if "sido_name" in dff.columns else "sido_code"
SIDO_LIST = sorted(dff[SIDO_COL].unique())

# =========================
# 3) trace 추가 함수
# =========================
def add_type_traces(fig: go.Figure, data: pd.DataFrame, name_prefix: str, visible: bool):
    """
    agencyType(지점/전시장, 대리점)별 trace를 추가.
    visible: 이 그룹 trace들의 초기 표시 여부
    """
    for agency_type, color in TYPE_COLOR.items():
        sub = data[data["agencyType"] == agency_type]
        fig.add_trace(
            go.Scattermapbox(
                lat=sub["lat"],
                lon=sub["lon"],
                mode="markers",
                marker=dict(size=9, color=color, opacity=0.85),
                text=sub["agencyName"],
                customdata=sub[
                    ["agencyType", "distance_km", "displayCarCount", "carMasterCount", "agencyAddress", "agencyTel"]
                ].values,
                hovertemplate=HOVER_TEMPLATE,
                name=f"{name_prefix}{agency_type}",
                visible=visible,
            )
        )

# =========================
# 4) Figure 구성 (전체 + 시도별)
# =========================
fig = go.Figure()

# (A) 전체: 2개 trace (초기 표시)
add_type_traces(fig, dff, name_prefix="전체 - ", visible=True)

# (B) 시도별: 각 시도마다 2개 trace (초기 숨김)
start_idx = len(fig.data)  # 전체 2개 다음 index
for s in SIDO_LIST:
    sub_s = dff[dff[SIDO_COL] == s]
    add_type_traces(fig, sub_s, name_prefix=f"{s} - ", visible=False)

# =========================
# 5) 드롭다운 버튼(visible 토글)
# =========================
buttons = []

# 전체 버튼: 처음 2개만 True
vis_all = [False] * len(fig.data)
vis_all[0] = True
vis_all[1] = True
buttons.append(dict(label="전체", method="update", args=[{"visible": vis_all}]))

# 시도 버튼: 시도마다 trace 2개씩 (전체 2개 이후부터)
for i, s in enumerate(SIDO_LIST):
    vis = [False] * len(fig.data)
    base = start_idx + i * 2
    vis[base] = True
    vis[base + 1] = True
    buttons.append(dict(label=str(s), method="update", args=[{"visible": vis}]))

# =========================
# 6) 레이아웃(가로 길게 + hover 글씨 검정)
# =========================
fig.update_layout(
    title="현대차 대리점/전시장 지도",
    width=1600,
    height=550,
    margin=dict(l=10, r=10, t=60, b=10),
    mapbox=dict(
        style="open-street-map",
        zoom=9,
        center=dict(lat=float(dff["lat"].mean()), lon=float(dff["lon"].mean())),
    ),
    updatemenus=[dict(
        type="dropdown",
        x=0.01, y=0.99,
        xanchor="left", yanchor="top",
        buttons=buttons
    )],
    legend=dict(orientation="h", y=1.02, x=1, xanchor="right", yanchor="bottom"),
    hoverlabel=dict(
        bgcolor="white",
        bordercolor="#D0D0D0",
        font=dict(
            family="Pretendard, Apple SD Gothic Neo, Malgun Gothic",
            size=14,
            color="black",  # ✅ 글씨 검정
        ),
        align="left",
    ),
)

fig.show(config={"scrollZoom": True, "displayModeBar": True})


C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\4082419382.py:71: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\4082419382.py:71: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\4082419382.py:71: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\4082419382.py:71: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\rosie\AppData\Local\Temp\ipykernel_27876\4082419382.py:71: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https:/